## BAGGING DECISION TREE

In [1]:
import pandas as pd

diabetesData = pd.read_csv('diabetes.csv')

In [2]:
diabetesData.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
colNames = []

for i in diabetesData:
    if i != 'Outcome':
        colNames.append(i)
    
X = pd.read_csv('diabetes.csv', usecols=colNames)
y = diabetesData.Outcome

In [85]:
import random
from sklearn.metrics import accuracy_score

# Bagging / Random sample with replacement
def GetTrainTestBagging(dataset, y, ratio):
    X_train = list()
    y_train = list()
    n_train = round(len(dataset) * ratio)
    train_index = list()
    
    # Create X_train, y_train with random with replacement
    while len(X_train) < n_train:
        X_temp = list()
        index = random.randrange(0, len(dataset))
        for i in range(0, len(dataset.columns)):
            X_temp.append(dataset.iloc[index][i])
            train_index.append(index)
        X_train.append(X_temp)
        y_train.append(y[index])
        
    X_test = list()
    y_test = list()
    train_index_unique = list() # Contain index of X_train in dataset
    
    for i in train_index:
        if i not in train_index_unique:
            train_index_unique.append(i)
            
    n_test = len(dataset) - len(train_index_unique)
    
    # Create X_test, y_test from dataset which not used in X_test, y_test
    for i in range(0, len(dataset)):
        if i not in train_index_unique:
            X_temp = list()
            for j in range(0, len(dataset.columns)):
                X_temp.append(dataset.iloc[i][j])
            X_test.append(X_temp)
            y_test.append(y[i])
          
    print('n_train : ', n_train, '\nn_test : ', n_test)
    return X_train, X_test, y_train, y_test

# Train + Predict
def PredictBagging(X_train, X_test, y_train, y_test, models):
    for model in models:
        TrainModel(model, X_train, y_train) # Train the classifier
    
    y_pred = [] # Contain list of y_pred of each trees
    for model in models:
        pred = model.predict(X_test)
        y_pred.append(pred)
    
    accuracyScore = [] # Contain accuracy of each model
    for i in range(0, len(y_pred)):
        accuracyScore.append(accuracy_score(y_test, y_pred[i]))
    
    print('Accuracy of each model:')
    for i in range(0, len(accuracyScore)):
        print('Model ',i+1,' : ',accuracyScore[i])
    
    y_pred2 = [] # Contain the most voted prediction
    for i in range(0, len(X_test)):
        temp = []
        for j in range (0, len(y_pred)):
            temp.append(y_pred[j][i])
        y_pred2.append(int(max(set(temp), key=temp.count)))
        
    return y_pred2

# Train classifier
def TrainModel(model, X_train, y_train):
    model.fit(X_train, y_train)

# Convert to DataFrame
def convert_to_dataframe(data, column):
    temp = []
    temp = pd.DataFrame(data, columns=column)
    return temp

#==============================================================================================================================#

from sklearn.model_selection import train_test_split

# VERSI 2 BEDA ALGORITMA, BELUM TAU MANA YANG BENAR
# Bagging versi 2
def PredictBagging2(dataset, y, train_size, models):
    print('n_dataset: ', len(dataset))    
    X_colnames = list() # Contain column names of X
    for i in dataset:
        X_colnames.append(i)
    
    # Split train and test
    X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=1-train_size, random_state=42)
    print('n_train: ', len(X_train))
    print('n_test: ', len(X_test))
    
    # Train each model with different subsample
    for model in models:
        X_train_bootstrap = list()
        y_train_bootstrap = list()
        for i in range(0, len(X_train)):
            index = random.randrange(0, len(X_train))
            X_temp = list()
            for j in range(0, len(X_train.columns)):
                X_temp.append(X_train.iloc[index][j])
            X_train_bootstrap.append(X_temp)
            y_train_bootstrap.append(y[index])
        X_train_bootstrap = convert_to_dataframe(X_train_bootstrap, column=X_colnames)
        y_train_bootstrap = convert_to_dataframe(y_train_bootstrap, column=['Outcome'])
        TrainModel(model, X_train_bootstrap, y_train_bootstrap)
    
    y_pred = list() # Contain y_pred of each model
    for model in models:
        temp = list()
        temp = model.predict(X_test)
        y_pred.append(temp)
        
    y_pred_most_voted = list() # Contain the most voted y_pred
    for i in range(0, len(X_test)):
        temp = list()
        for j in range(0, len(y_pred)):
            temp.append(y_pred[j][i])
        y_pred_most_voted.append(int(max(set(temp), key=temp.count)))
    
    print('\nAccuracy of each model:')
    for i in range(0, len(models)):
        print('Model ', i+1, ': ', accuracy_score(y_test, y_pred[i]))
        
    print('\nEnsemble accuracy: ', accuracy_score(y_test, y_pred_most_voted))
    return y_pred_most_voted

In [20]:
X_train, X_test, y_train, y_test = GetTrainTestBagging(X, y, ratio=0.7)

X_train = convert_to_dataframe(X_train, colNames)
X_test = convert_to_dataframe(X_test, colNames)
y_train = convert_to_dataframe(y_train, column=['Outcome'])
y_test = convert_to_dataframe(y_test, column=['Outcome'])

n_train :  538 
n_test :  370


In [21]:
from sklearn.tree import DecisionTreeClassifier

model1 = DecisionTreeClassifier(criterion='gini', max_depth=10)
model2 = DecisionTreeClassifier(criterion='entropy', max_depth=10)
model3 = DecisionTreeClassifier(criterion='gini', max_depth=6)
model4 = DecisionTreeClassifier(criterion='entropy', max_depth=6)
model5 = DecisionTreeClassifier(criterion='gini', max_depth=15)
model6 = DecisionTreeClassifier(criterion='entropy', max_depth=15)
model7 = DecisionTreeClassifier(criterion='gini', max_depth=8)
model8 = DecisionTreeClassifier(criterion='entropy', max_depth=8)
model9 = DecisionTreeClassifier(criterion='gini')
model10 = DecisionTreeClassifier(criterion='entropy')
    
models = [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10]

y_pred = PredictBagging(X_train, X_test, y_train, y_test, models)

Accuracy of each model:
Model  1  :  0.6837837837837838
Model  2  :  0.7162162162162162
Model  3  :  0.6918918918918919
Model  4  :  0.7081081081081081
Model  5  :  0.6864864864864865
Model  6  :  0.7135135135135136
Model  7  :  0.6864864864864865
Model  8  :  0.7135135135135136
Model  9  :  0.6837837837837838
Model  10  :  0.7135135135135136


In [96]:
print('Ensemble accuracy : ', accuracy_score(y_test, y_pred))

Ensemble accuracy :  0.727027027027027


In [101]:
# VERSI 2
y_pred_versi2 = PredictBagging2(X, y, 0.7, models)

n_dataset:  768
n_train:  537
n_test:  231

Accuracy of each model:
Model  1 :  0.5194805194805194
Model  2 :  0.5670995670995671
Model  3 :  0.5454545454545454
Model  4 :  0.6363636363636364
Model  5 :  0.5800865800865801
Model  6 :  0.5670995670995671
Model  7 :  0.5021645021645021
Model  8 :  0.6536796536796536
Model  9 :  0.5238095238095238
Model  10 :  0.5151515151515151

Ensemble accuracy:  0.6233766233766234
